In [164]:
import warnings
warnings.filterwarnings('ignore')
!pip install gensim
!pip install nltk

In [165]:
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from sklearn.model_selection import train_test_split

import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

plt.style.use('seaborn')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [166]:
# Google TPU einrichten (muss in den Notebook-Einstelllungen aktiviert sein)
# bei Nichtnutzung einfach die Zeile "with tpu_strategy.scope():" löschen
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.34.98.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.34.98.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [167]:
#Dictionary mit allen .json
FILES = {1: 'Gangart.json',
        2 : 'Vegan.json',
        3 : 'Laender.json',
        4 : 'Mahlzeit.json'}    

In [168]:
CURRENT_FILE = FILES[1]

#Nur Glove (Zubereitungshinweise) verwenden oder über Multiple Input auch die Zutaten?
BOTH_NEURAL_NETWORKS = False

#Minimale Länge aller Directions
MINLEN_DIRECTIONS = 106

# !hier Pfad angeben, wo die Daten liegen!
DATA_PATH = 'drive/MyDrive/Colab Notebooks/Data/'

In [169]:
#Load Data
with open(DATA_PATH + CURRENT_FILE) as file:
    data = json.load(file)

In [170]:
#Directions aufbereiten
def preprocess_text(sen):

    # Punkte und Zahlen eliminieren
    sen = re.sub('[^a-zA-Z]', ' ', sen)

    # Einzelne Characters entfernen
    sen = re.sub(r"\s+[a-zA-Z]\s+", ' ', sen)

    # Mehrere Leerzeichen entfernen
    sen = re.sub(r'\s+', ' ', sen)

    #Stopwords entfernen
    sen = ' '.join(filter(lambda word: word.lower() not in stopwords.words('english'),  sen.split()))

    return sen

TAG_RE = re.compile(r'<[^>]+>')

In [171]:
#Input und Output Vektor
inputData = []
labels = []

#Dictionaries
index_Kategorien = dict()
index_Zutaten = dict()

#Create Dict for Ingredients and Categories and Ditections with index as value
def createDict():
    #Categories and Ingredients
    counterOne = 0
    counterTwo = 0
    counterThree = 0
    for recipe in data:
        ingredients = recipe['ingredients']
        category = recipe['category']

        for ing in ingredients:
            if ing not in index_Zutaten:
                index_Zutaten[ing] = counterOne
                counterOne +=1
        
        if category not in index_Kategorien:
            index_Kategorien[category] = counterTwo
            counterTwo +=1 
            
            
    
#Create Input Vector and Output Vector (OneHot)
def createDataAndLabel():
    for recipe in data:
        
        #Input Vector
        ingredients = recipe['ingredients']
        category = recipe['category']
        anweisungen = recipe['directions_string']
        
        inputVectorZutaten = np.zeros(len(index_Zutaten))
        inputVectorAnweisungen = []
        
        #InputVector Zutaten
        for zutat in ingredients:
            if zutat in index_Zutaten:
                inputVectorZutaten[index_Zutaten[zutat]] = 1
        
        inputData.append([inputVectorZutaten,preprocess_text(anweisungen)])
                
        #Output Vector
        outputVector = np.zeros(len(index_Kategorien))
        outputVector[index_Kategorien[category]] = 1
        labels.append(outputVector)

In [172]:
with tpu_strategy.scope():
  createDict()
  createDataAndLabel()

In [173]:
def splitData():
  train_input, test_input, train_label, test_label = train_test_split(inputData, labels, test_size=0.2, random_state=42)
  #Input Vektor für Zutaten und Anweisungen trennen
  train_input_zutaten = []
  train_input_anweisungen = []
  test_input_zutaten = []
  test_input_anweisungen = []

  for entry in train_input:
      train_input_zutaten.append(entry[0])
      train_input_anweisungen.append(entry[1])
      
  for entry in test_input:
      test_input_zutaten.append(entry[0])
      test_input_anweisungen.append(entry[1])
      
  #Als numpy-array
  train_label = np.array(train_label)
  test_label = np.array(test_label)
  train_input_zutaten = np.array(train_input_zutaten)
  test_input_zutaten = np.array(test_input_zutaten)

  return train_label, test_label, train_input_zutaten, test_input_zutaten,train_input_anweisungen, test_input_anweisungen

In [174]:
train_label, test_label, train_input_zutaten, test_input_zutaten,train_input_anweisungen,test_input_anweisungen = splitData()

In [175]:
def tokenize_directions(train_input_anweisungen, test_input_anweisungen):
  #Anweisungen tokenizen und indizieren
  tokenizer = Tokenizer(num_words=500)
  tokenizer.fit_on_texts(train_input_anweisungen)

  train_input_anweisungen = tokenizer.texts_to_sequences(train_input_anweisungen)
  test_input_anweisungen = tokenizer.texts_to_sequences(test_input_anweisungen)
  
  return train_input_anweisungen, test_input_anweisungen, tokenizer

In [176]:
train_input_anweisungen, test_input_anweisungen, tokenizer = tokenize_directions(train_input_anweisungen, test_input_anweisungen)

In [177]:
def findMaxLength(lst): 
    maxList = max(lst, key = lambda i: len(i)) 
    maxLength = len(maxList) 
    return maxLength

In [178]:
#Anweisungen padden
MAXLEN = min(MINLEN_DIRECTIONS,max(findMaxLength(train_input_anweisungen),findMaxLength(test_input_anweisungen)))

train_input_anweisungen = pad_sequences(train_input_anweisungen, padding = 'post', maxlen = MAXLEN)
test_input_anweisungen = pad_sequences(test_input_anweisungen, padding = 'post', maxlen = MAXLEN)

In [179]:
VOCAB_SIZE = len(tokenizer.word_index) + 1

In [180]:
def create_glove_embedding():
  # Create GloVe Embedding Dictionary
  embeddings_dictionary = dict()
  glove_file = open(DATA_PATH + '/glove.6B.100d.txt', encoding = 'utf8')

  for line in glove_file:
      records = line.split()
      word = records[0]
      vector_dimensions = np.asarray(records[1:], dtype='float32')
      embeddings_dictionary[word] = vector_dimensions
  glove_file.close()

  # Create Embedding Matrix
  embedding_matrix = np.zeros((VOCAB_SIZE, 100))

  for word, index in tokenizer.word_index.items():
      embedding_vector = embeddings_dictionary.get(word)
      if embedding_vector is not None:
          embedding_matrix[index] = embedding_vector
  return embedding_matrix

In [181]:
with tpu_strategy.scope():
  embedding_matrix = create_glove_embedding()

In [182]:
#Hyperparamter festlegen
INPUT_SHAPE_ZUTATEN = len(index_Zutaten)
INPUT_SHAPE_ANWEISUNGEN = MAXLEN
DROPOUT_RATE = 0.5
OUTPUT_SHAPE = len(index_Kategorien)
OUTPUT_SHAPE_ZUTATEN = 32
OUTPUT_SHAPE_ANWEISUNGEN = 32
EPOCHS = 10
BATCH_SIZE = 32
LOSS_FUNCTION = 'categorical_crossentropy'
OPTIMIZER = 'adam'

In [183]:
#Methode zur Erstellung des Zutaten Neuronalen Netzes (MLP)
def createZutatenNN(input_shape,p_dropout,output_shape):
    model = Sequential()
    model.add(Dense(128, input_dim=input_shape, activation='relu'))
    model.add(Dropout(p_dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p_dropout))
    model.add(Dense(output_shape, activation='softmax'))
    
    return model

In [184]:
#Methode zur Erstellung des Anweisungen Convolutional Neuronalen Netzes
def createAnweisungenNN(input_shape,p_dropout,output_shape):
    modelCNN = Sequential()
    embedding_layerC = Embedding(VOCAB_SIZE, 100, weights = [embedding_matrix], input_length = input_shape, trainable = False) 

    modelCNN.add(embedding_layerC)
    modelCNN.add(Conv1D(128, 10, activation='relu', input_shape=(None, MAXLEN, 100)))
    modelCNN.add(Dropout(p_dropout))
    modelCNN.add(MaxPooling1D(pool_size = 4))
    modelCNN.add(Conv1D(256, 5, activation='relu'))
    modelCNN.add(GlobalMaxPooling1D())
    modelCNN.add(Dropout(p_dropout))
    modelCNN.add(Dense(output_shape, activation = 'relu'))
    
    return modelCNN

In [185]:
def createCombinedNN():
    #Beide Neuronalen Netze erstellen
    zutatenNN = createZutatenNN(input_shape=INPUT_SHAPE_ZUTATEN,p_dropout=DROPOUT_RATE,output_shape=OUTPUT_SHAPE_ZUTATEN)
    anweisungenNN = createAnweisungenNN(input_shape=INPUT_SHAPE_ANWEISUNGEN,p_dropout=DROPOUT_RATE,
                                      output_shape=OUTPUT_SHAPE_ANWEISUNGEN)
    #Kombiniertes Neuronales Netz erstellen
    combinedInput = concatenate([zutatenNN.output, anweisungenNN.output])
    x = Dense(16, activation="relu")(combinedInput)
    x = Dense(OUTPUT_SHAPE, activation="softmax")(x)
    model = Model(inputs=[zutatenNN.input, anweisungenNN.input], outputs=x)

    return model

In [186]:
def create_model(summary=False):
  if BOTH_NEURAL_NETWORKS:
    model = createCombinedNN()
  else:
    model = createZutatenNN(input_shape=INPUT_SHAPE_ZUTATEN,p_dropout=DROPOUT_RATE,output_shape=OUTPUT_SHAPE)
   
  if summary:
    model.summary()

  model.compile(loss=LOSS_FUNCTION, optimizer=OPTIMIZER,metrics=['accuracy'])

  return model

In [187]:
def fit_model():
  model = create_model(True)
  #Model trainieren
  print('Trainieren auf ' + str(len(train_input_anweisungen)) + " Datensätzen.")
  print('Testen auf ' + str(len(test_input_anweisungen)) + " Datensätzen. \n" )

  if BOTH_NEURAL_NETWORKS:
    history = model.fit(x=[train_input_zutaten,train_input_anweisungen], y=train_label,
          validation_data=([test_input_zutaten,test_input_anweisungen], test_label),
          epochs=EPOCHS, batch_size=BATCH_SIZE)
  else:
    history = model.fit(train_input_zutaten,train_label,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_data=(test_input_zutaten, test_label))


In [ ]:
with tpu_strategy.scope():
  fit_model()



Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 128)               137984    
_________________________________________________________________
dropout_26 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_27 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 7)                 455       
Total params: 146,695
Trainable params: 146,695
Non-trainable params: 0
_________________________________________________________________
Trainieren auf 6424 Datensätzen.
Testen auf 1606 Datensätzen. 

Epoch 1/10
201/201 [=============================

In [ ]:
len(train_input_zutaten[0])

In [ ]:
def visualize(h):
    plt.plot(h.history['accuracy'], color = 'blue')
    plt.plot(h.history['val_accuracy'], color = 'red')
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()

    print('')

    plt.plot(h.history['loss'], color = 'blue')
    plt.plot(h.history['val_loss'], color = 'red')

    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()

In [ ]:
visualize(history)

In [ ]:
tf.__version__